In [2]:
!pip install tensorflow==2.5
!pip install --upgrade tensorflow-hub

  Using cached tensorflow-2.5.0-cp37-cp37m-manylinux2010_x86_64.whl (454.3 MB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached h5py-3.1.0-cp37-cp37m-manylinux1_x86_64.whl (4.0 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl
  Using cached grpcio-1.34.1-cp37-cp37m-manylinux2014_x86_64.whl (4.0 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached google_pasta-

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import torch
from torch import nn

2021-11-23 11:06:10.747089: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

2021-11-23 11:06:20.152996: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-11-23 11:06:20.364508: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-11-23 11:06:20.367583: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:07:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-11-23 11:06:20.370515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBan

In [5]:
for k in vars(hub_module).keys():
  print(k)

_self_setattr_tracking
_self_unconditional_checkpoint_dependencies
_self_unconditional_dependency_names
_self_unconditional_deferred_dependencies
_self_update_uid
_self_name_based_restores
_self_saveable_object_factories
variables
save_counter
signatures
__call__
graph_debug_info
tensorflow_version
tensorflow_git_version
_is_hub_module_v1


In [6]:
hub_module._is_hub_module_v1

False

In [7]:
DICT = {v.name:v for v in sorted(hub_module.variables, key=lambda v:v.name)}

In [8]:
for idx, (k, v) in enumerate(DICT.items()):
  print(f"{idx:3d}", k, v.shape)

  0 InceptionV3/Conv2d_1a_3x3/BatchNorm/beta:0 (32,)
  1 InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean:0 (32,)
  2 InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_variance:0 (32,)
  3 InceptionV3/Conv2d_1a_3x3/weights:0 (3, 3, 3, 32)
  4 InceptionV3/Conv2d_2a_3x3/BatchNorm/beta:0 (32,)
  5 InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_mean:0 (32,)
  6 InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_variance:0 (32,)
  7 InceptionV3/Conv2d_2a_3x3/weights:0 (3, 3, 32, 32)
  8 InceptionV3/Conv2d_2b_3x3/BatchNorm/beta:0 (64,)
  9 InceptionV3/Conv2d_2b_3x3/BatchNorm/moving_mean:0 (64,)
 10 InceptionV3/Conv2d_2b_3x3/BatchNorm/moving_variance:0 (64,)
 11 InceptionV3/Conv2d_2b_3x3/weights:0 (3, 3, 32, 64)
 12 InceptionV3/Conv2d_3b_1x1/BatchNorm/beta:0 (80,)
 13 InceptionV3/Conv2d_3b_1x1/BatchNorm/moving_mean:0 (80,)
 14 InceptionV3/Conv2d_3b_1x1/BatchNorm/moving_variance:0 (80,)
 15 InceptionV3/Conv2d_3b_1x1/weights:0 (1, 1, 64, 80)
 16 InceptionV3/Conv2d_4a_3x3/BatchNorm/beta:0 (192,)
 17 InceptionV3/Con

In [9]:
from transform26 import Ghiasi
TRANSFORMER = Ghiasi()

In [10]:
eps = 1e-3

# Dirty

# First 3 conv

In [11]:
for idx, l in enumerate(TRANSFORMER.layers[:3],1):
  conv_weight = DICT[f"transformer/contract/conv{idx}/weights:0"].numpy()
  BN_beta = DICT[f"transformer/contract/conv{idx}/BatchNorm/beta:0"].numpy()
  BN_moving_mean = DICT[f"transformer/contract/conv{idx}/BatchNorm/moving_mean:0"].numpy()
  BN_moving_variance = DICT[f"transformer/contract/conv{idx}/BatchNorm/moving_variance:0"].numpy()
  BN_moving_std = (eps+BN_moving_variance)**0.5
  
  weight = nn.Parameter(
    torch.einsum(
      "abio,o->oiab",
      torch.from_numpy(conv_weight),
      torch.from_numpy(BN_moving_std),
    )
  )
  bias = torch.from_numpy((BN_beta - BN_moving_mean/BN_moving_std))
  
  l.conv.load_state_dict({
    "weight":weight*255 if idx==1 else weight,
    "bias":bias,
  })

# Last 3 Conv

In [12]:
DICT[f"transformer/expand/conv{idx}/conv/weights:0"].shape

TensorShape([9, 9, 32, 3])

In [13]:
for _ in TRANSFORMER.layers[-3].named_parameters():
  print(_[0], _[1].shape)

conv.weight torch.Size([64, 128, 3, 3])
fc_beta.weight torch.Size([64, 768])
fc_beta.bias torch.Size([64])
fc_gamma.weight torch.Size([64, 768])
fc_gamma.bias torch.Size([64])


In [14]:
for idx, l in enumerate(TRANSFORMER.layers[-3:],1):
  conv_weight = DICT[f"transformer/expand/conv{idx}/conv/weights:0"].numpy()
  conv_weight = nn.Parameter(
    torch.einsum(
      "abio->oiab",
      torch.from_numpy(conv_weight),
    )
  )
  w1 = torch.from_numpy(DICT[f"style_params/transformer/expand/conv{idx}/conv/StyleNorm/Conv/weights:0"].numpy()).squeeze().permute(1,0)
  b1 = torch.from_numpy(DICT[f"style_params/transformer/expand/conv{idx}/conv/StyleNorm/Conv/biases:0"].numpy())
  w2 = torch.from_numpy(DICT[f"style_params/transformer/expand/conv{idx}/conv/StyleNorm/Conv_1/weights:0"].numpy()).squeeze().permute(1,0)
  b2 = torch.from_numpy(DICT[f"style_params/transformer/expand/conv{idx}/conv/StyleNorm/Conv_1/biases:0"].numpy())
  
  l.load_state_dict({
    "conv.weight":conv_weight,
    "fc_beta.weight":w1,
    "fc_beta.bias":b1,
    "fc_gamma.weight":w2,
    "fc_gamma.bias":b2,
  })

# Middle Residual Layers

In [15]:
for n, _ in TRANSFORMER.layers[3].named_parameters():
  print(f"\"{n}\": ,")

"conv1.weight": ,
"fc_beta1.weight": ,
"fc_beta1.bias": ,
"fc_gamma1.weight": ,
"fc_gamma1.bias": ,
"fc_beta2.weight": ,
"fc_beta2.bias": ,
"fc_gamma2.weight": ,
"fc_gamma2.bias": ,
"conv2.weight": ,


In [16]:
for idx, l in enumerate(TRANSFORMER.layers[3:-3],1):
# 292 style_params/transformer/residual/residual1/conv1/StyleNorm/Conv/biases:0 (128,)
# 293 style_params/transformer/residual/residual1/conv1/StyleNorm/Conv/weights:0 (1, 1, 768, 128)
# 294 style_params/transformer/residual/residual1/conv1/StyleNorm/Conv_1/biases:0 (128,)
# 295 style_params/transformer/residual/residual1/conv1/StyleNorm/Conv_1/weights:0 (1, 1, 768, 128)
# 296 style_params/transformer/residual/residual1/conv2/StyleNorm/Conv/biases:0 (128,)
# 297 style_params/transformer/residual/residual1/conv2/StyleNorm/Conv/weights:0 (1, 1, 768, 128)
# 298 style_params/transformer/residual/residual1/conv2/StyleNorm/Conv_1/biases:0 (128,)
# 299 style_params/transformer/residual/residual1/conv2/StyleNorm/Conv_1/weights:0 (1, 1, 768, 128)

# 467 transformer/residual/residual1/conv1/weights:0 (3, 3, 128, 128)
# 468 transformer/residual/residual1/conv2/weights:0 (3, 3, 128, 128)

  conv1_weight = DICT[f"transformer/residual/residual{idx}/conv1/weights:0"].numpy()
  conv1_weight = nn.Parameter(
    torch.einsum(
      "abio->oiab",
      torch.from_numpy(conv1_weight),
    )
  )
  conv2_weight = DICT[f"transformer/residual/residual{idx}/conv2/weights:0"].numpy()
  conv2_weight = nn.Parameter(
    torch.einsum(
      "abio->oiab",
      torch.from_numpy(conv2_weight),
    )
  )
  
  fc_beta1_weight = torch.from_numpy(DICT[f"style_params/transformer/residual/residual{idx}/conv1/StyleNorm/Conv/weights:0"].numpy())
  fc_beta1_bias = torch.from_numpy(DICT[f"style_params/transformer/residual/residual{idx}/conv1/StyleNorm/Conv/biases:0"].numpy())
  fc_gamma1_weight = torch.from_numpy(DICT[f"style_params/transformer/residual/residual{idx}/conv1/StyleNorm/Conv_1/weights:0"].numpy())
  fc_gamma1_bias = torch.from_numpy(DICT[f"style_params/transformer/residual/residual{idx}/conv1/StyleNorm/Conv_1/biases:0"].numpy())
  fc_beta2_weight = torch.from_numpy(DICT[f"style_params/transformer/residual/residual{idx}/conv2/StyleNorm/Conv/weights:0"].numpy())
  fc_beta2_bias = torch.from_numpy(DICT[f"style_params/transformer/residual/residual{idx}/conv2/StyleNorm/Conv/biases:0"].numpy())
  fc_gamma2_weight = torch.from_numpy(DICT[f"style_params/transformer/residual/residual{idx}/conv2/StyleNorm/Conv_1/weights:0"].numpy())
  fc_gamma2_bias = torch.from_numpy(DICT[f"style_params/transformer/residual/residual{idx}/conv2/StyleNorm/Conv_1/biases:0"].numpy())
  
  l.load_state_dict({
    "conv1.weight": conv1_weight,
    "fc_beta1.weight": fc_beta1_weight.squeeze().permute(1,0),
    "fc_beta1.bias": fc_beta1_bias,
    "fc_gamma1.weight": fc_gamma1_weight.squeeze().permute(1,0),
    "fc_gamma1.bias": fc_gamma1_bias,
    "fc_beta2.weight": fc_beta2_weight.squeeze().permute(1,0),
    "fc_beta2.bias": fc_beta2_bias,
    "fc_gamma2.weight": fc_gamma2_weight.squeeze().permute(1,0),
    "fc_gamma2.bias": fc_gamma2_bias,
    "conv2.weight": conv2_weight,
  })
  pass

In [17]:
for idx, (n, _) in enumerate(TRANSFORMER.named_parameters()):
  print(idx, n)

0 layers.0.conv.weight
1 layers.0.conv.bias
2 layers.1.conv.weight
3 layers.1.conv.bias
4 layers.2.conv.weight
5 layers.2.conv.bias
6 layers.3.conv1.weight
7 layers.3.fc_beta1.weight
8 layers.3.fc_beta1.bias
9 layers.3.fc_gamma1.weight
10 layers.3.fc_gamma1.bias
11 layers.3.fc_beta2.weight
12 layers.3.fc_beta2.bias
13 layers.3.fc_gamma2.weight
14 layers.3.fc_gamma2.bias
15 layers.3.conv2.weight
16 layers.4.conv1.weight
17 layers.4.fc_beta1.weight
18 layers.4.fc_beta1.bias
19 layers.4.fc_gamma1.weight
20 layers.4.fc_gamma1.bias
21 layers.4.fc_beta2.weight
22 layers.4.fc_beta2.bias
23 layers.4.fc_gamma2.weight
24 layers.4.fc_gamma2.bias
25 layers.4.conv2.weight
26 layers.5.conv1.weight
27 layers.5.fc_beta1.weight
28 layers.5.fc_beta1.bias
29 layers.5.fc_gamma1.weight
30 layers.5.fc_gamma1.bias
31 layers.5.fc_beta2.weight
32 layers.5.fc_beta2.bias
33 layers.5.fc_gamma2.weight
34 layers.5.fc_gamma2.bias
35 layers.5.conv2.weight
36 layers.6.conv1.weight
37 layers.6.fc_beta1.weight
38 layers

In [19]:
torch.save(
  TRANSFORMER.state_dict(),
  "weight/Ghiasi26.pth"
)

# Go build style predictor

In [20]:
from stylePredictor26 import StylePredictor

In [21]:
SP = StylePredictor()

In [22]:
from itertools import chain
LIST = [
  n for (n, _) in
  chain(SP.named_parameters(), SP.named_buffers())
]
for idx, n in enumerate(
  filter(
    lambda name: ("num_batches_tracked" not in name and "bn.weight" not in name),
    sorted(LIST)
  )
):
  print(f"{idx:3d}", n, tuple(SP.get_parameter(n).shape))

  0 Conv2d_1a_3x3.conv.bias (32,)
  1 Conv2d_1a_3x3.conv.weight (32, 3, 3, 3)
  2 Conv2d_2a_3x3.conv.bias (32,)
  3 Conv2d_2a_3x3.conv.weight (32, 32, 3, 3)
  4 Conv2d_2b_3x3.conv.bias (64,)
  5 Conv2d_2b_3x3.conv.weight (64, 32, 3, 3)
  6 Conv2d_3b_1x1.conv.bias (80,)
  7 Conv2d_3b_1x1.conv.weight (80, 64, 1, 1)
  8 Conv2d_4a_3x3.conv.bias (192,)
  9 Conv2d_4a_3x3.conv.weight (192, 80, 3, 3)
 10 Mixed_5b.branch1x1.conv.bias (64,)
 11 Mixed_5b.branch1x1.conv.weight (64, 192, 1, 1)
 12 Mixed_5b.branch3x3dbl_1.conv.bias (64,)
 13 Mixed_5b.branch3x3dbl_1.conv.weight (64, 192, 1, 1)
 14 Mixed_5b.branch3x3dbl_2.conv.bias (96,)
 15 Mixed_5b.branch3x3dbl_2.conv.weight (96, 64, 3, 3)
 16 Mixed_5b.branch3x3dbl_3.conv.bias (96,)
 17 Mixed_5b.branch3x3dbl_3.conv.weight (96, 96, 3, 3)
 18 Mixed_5b.branch5x5_1.conv.bias (48,)
 19 Mixed_5b.branch5x5_1.conv.weight (48, 192, 1, 1)
 20 Mixed_5b.branch5x5_2.conv.bias (64,)
 21 Mixed_5b.branch5x5_2.conv.weight (64, 48, 5, 5)
 22 Mixed_5b.branch_pool.conv

```
16 InceptionV3/Conv2d_4a_3x3/BatchNorm/beta:0 (192,)
17 InceptionV3/Conv2d_4a_3x3/BatchNorm/moving_mean:0 (192,)
18 InceptionV3/Conv2d_4a_3x3/BatchNorm/moving_variance:0 (192,)
19 InceptionV3/Conv2d_4a_3x3/weights:0 (3, 3, 80, 192)
20 InceptionV3/Mixed_5b/Branch_0/Conv2d_0a_1x1/BatchNorm/beta:0 (64,)
21 InceptionV3/Mixed_5b/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_mean:0 (64,)
22 InceptionV3/Mixed_5b/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_variance:0 (64,)
23 InceptionV3/Mixed_5b/Branch_0/Conv2d_0a_1x1/weights:0 (1, 1, 192, 64)
```

In [23]:
from collections import defaultdict

In [24]:
D_pridctor_pre_register = defaultdict(dict)

In [25]:
for name, v in sorted(DICT.items()):
  if "InceptionV3" not in name:
    continue
  
  splited_string = name.split("/")
  last_idx = -2 if "BatchNorm" in name else -1
  base_name = ".".join(splited_string[1:last_idx])
  surffix = splited_string[-1]
  print(base_name, surffix)
  D_pridctor_pre_register[base_name][surffix[:-2]] = v.numpy()

Conv2d_1a_3x3 beta:0
Conv2d_1a_3x3 moving_mean:0
Conv2d_1a_3x3 moving_variance:0
Conv2d_1a_3x3 weights:0
Conv2d_2a_3x3 beta:0
Conv2d_2a_3x3 moving_mean:0
Conv2d_2a_3x3 moving_variance:0
Conv2d_2a_3x3 weights:0
Conv2d_2b_3x3 beta:0
Conv2d_2b_3x3 moving_mean:0
Conv2d_2b_3x3 moving_variance:0
Conv2d_2b_3x3 weights:0
Conv2d_3b_1x1 beta:0
Conv2d_3b_1x1 moving_mean:0
Conv2d_3b_1x1 moving_variance:0
Conv2d_3b_1x1 weights:0
Conv2d_4a_3x3 beta:0
Conv2d_4a_3x3 moving_mean:0
Conv2d_4a_3x3 moving_variance:0
Conv2d_4a_3x3 weights:0
Mixed_5b.Branch_0.Conv2d_0a_1x1 beta:0
Mixed_5b.Branch_0.Conv2d_0a_1x1 moving_mean:0
Mixed_5b.Branch_0.Conv2d_0a_1x1 moving_variance:0
Mixed_5b.Branch_0.Conv2d_0a_1x1 weights:0
Mixed_5b.Branch_1.Conv2d_0a_1x1 beta:0
Mixed_5b.Branch_1.Conv2d_0a_1x1 moving_mean:0
Mixed_5b.Branch_1.Conv2d_0a_1x1 moving_variance:0
Mixed_5b.Branch_1.Conv2d_0a_1x1 weights:0
Mixed_5b.Branch_1.Conv2d_0b_5x5 beta:0
Mixed_5b.Branch_1.Conv2d_0b_5x5 moving_mean:0
Mixed_5b.Branch_1.Conv2d_0b_5x5 movi

In [26]:
len(D_pridctor_pre_register)

70

In [27]:
# D_pridctor = defaultdict(lambda:None)

In [28]:
# for name, varaibles in D_pridctor_pre_register.items():
#   run_std = ((eps+varaibles["moving_variance"])**0.5)
#   D_pridctor[name+".conv.weight"] = varaibles["weights"].transpose(3,2,0,1)/run_std.reshape(-1,1,1,1)
#   D_pridctor[name+".conv.bias"] = varaibles["beta"]-varaibles["moving_mean"]/run_std

In [29]:
D_predictor_final = {}

In [30]:
for (_, varaibles), (name, _) in zip(D_pridctor_pre_register.items(), list(SP.named_parameters())[::2]):
  run_std = ((eps+varaibles["moving_variance"])**0.5)
  D_predictor_final[name[:-7]+".weight"] = torch.from_numpy(varaibles["weights"].transpose(3,2,0,1)/run_std.reshape(-1,1,1,1))
  D_predictor_final[name[:-7]+".bias"] = torch.from_numpy(varaibles["beta"]-varaibles["moving_mean"]/run_std)

In [31]:
SP.load_state_dict(D_predictor_final)

<All keys matched successfully>

In [32]:
for name, m in SP.named_modules():
  if isinstance(m, torch.nn.modules.conv.Conv2d):
    print(m.in_channels, m.out_channels, m.weight.shape)

3 32 torch.Size([32, 3, 3, 3])
32 32 torch.Size([32, 32, 3, 3])
32 64 torch.Size([64, 32, 3, 3])
64 80 torch.Size([80, 64, 1, 1])
80 192 torch.Size([192, 80, 3, 3])
192 64 torch.Size([64, 192, 1, 1])
192 48 torch.Size([48, 192, 1, 1])
48 64 torch.Size([64, 48, 5, 5])
192 64 torch.Size([64, 192, 1, 1])
64 96 torch.Size([96, 64, 3, 3])
96 96 torch.Size([96, 96, 3, 3])
192 32 torch.Size([32, 192, 1, 1])
256 64 torch.Size([64, 256, 1, 1])
256 48 torch.Size([48, 256, 1, 1])
48 64 torch.Size([64, 48, 5, 5])
256 64 torch.Size([64, 256, 1, 1])
64 96 torch.Size([96, 64, 3, 3])
96 96 torch.Size([96, 96, 3, 3])
256 64 torch.Size([64, 256, 1, 1])
288 64 torch.Size([64, 288, 1, 1])
288 48 torch.Size([48, 288, 1, 1])
48 64 torch.Size([64, 48, 5, 5])
288 64 torch.Size([64, 288, 1, 1])
64 96 torch.Size([96, 64, 3, 3])
96 96 torch.Size([96, 96, 3, 3])
288 64 torch.Size([64, 288, 1, 1])
288 384 torch.Size([384, 288, 3, 3])
288 64 torch.Size([64, 288, 1, 1])
64 96 torch.Size([96, 64, 3, 3])
96 96 torch.S

In [33]:
torch.save(SP.state_dict(), "weight/predictor26.pth")